In [1]:
import pandas as pd
import numpy as np
import ast
import decimal
import psycopg2
import json

In [2]:
pd.set_option('display.max_rows', 3000)

In [3]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password=""
)

In [4]:
# Open a cursor to perform database operations
cur = conn.cursor()

In [5]:
users = []

In [6]:
def insert_user_info(tweet):
    try:
        
#         cur.execute("""
#         INSERT INTO twitter_users_partitioned 
#         (user_id, name, screen_name, date, twitter_join_date, location, description, 
#         verified, followers_count, friends_count, listed_count, favourites_count, language)
#         VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#     """, (tweet['user']['id'], tweet['user']['name'], tweet['screen_name'], tweet['created_at'], 
#           tweet['user']['created_at'], tweet['user']['location'], tweet['user']['description'], 
#           tweet['user']['verified'], tweet['user']['followers_count'], tweet['user']['friends_count'], 
#           tweet['user']['listed_count'], tweet['user']['favourites_count'], tweet['user']['lang']))
        
        users.append({'user_id': tweet['user']['id'], 
                      'name': tweet['user']['name'], 
                      'screen_name': tweet['user']['screen_name'], 
                      'date': tweet['created_at'],
                      'twitter_join_date': tweet['user']['created_at'], 
                      'location': tweet['user']['location'], 
                      'description': tweet['user']['description'], 
                      'verified': tweet['user']['verified'], 
                      'followers_count': tweet['user']['followers_count'], 
                      'friends_count': tweet['user']['friends_count'],
                      'listed_count': tweet['user']['listed_count'], 
                      'favourites_count': tweet['user']['favourites_count'],
                      'language': tweet['user']['lang']})
#         conn.commit()
#         cur.close()
#         conn.close()

    except Exception as e:
#         conn.rollback()
#         cur.close()
#         conn.close()
        print(e)

In [7]:
def load_data(file_path):
    # Load the JSON data from file
    with open(file_path, "r") as f:
        for line in f:
            try:
                tweet = json.loads(line)
                # insert user entry into database
                insert_user_info(tweet)
                # if there is a retweet, get original user from retweet
                if (tweet['text'].startswith('RT')):
                    original_tweet = tweet["retweeted_status"]
                    insert_user_info(original_tweet)
            except:
                # if there is an error loading the json of the tweet, skip
                continue

In [8]:
load_data("../corona-out-2")

In [9]:
load_data("../corona-out-3")

In [10]:
# Create a pandas DataFrame from the parsed data
df_users = pd.DataFrame(users)

In [11]:
df_users.head()

,user_id,name,screen_name,date,twitter_join_date,location,description,verified,followers_count,friends_count,listed_count,favourites_count,language
0,1242817830946508801,juwelz v,juwelz_v,Sun Apr 12 18:27:25 +0000 2020,Wed Mar 25 14:17:28 +0000 2020,"Lower East Side, Manhattan",Event Lyfe LLC .. Brand Ambassador: #visionary...,False,43,118,0,722,None
1,16144221,NUFF,nuffsaidny,Sun Apr 12 16:48:01 +0000 2020,Fri Sep 05 14:28:41 +0000 2008,None,instagram: @nuffsaidny 🇹🇹,False,17112,1515,874,15790,None
2,1225145123920588805,efe09,efe0927183508,Sun Apr 12 18:27:25 +0000 2020,Wed Feb 05 19:52:38 +0000 2020,None,Allah'ın en değerli eseri insandır.\nCanı yana...,False,653,983,0,1255,None
3,1087735689091928064,Karanfil Lale,lale_karanfil,Sun Apr 12 18:02:41 +0000 2020,Tue Jan 22 15:36:12 +0000 2019,None,None,False,897,1120,1,2776,None
4,101007632,Ravin Gupta,IamRaavin,Sun Apr 12 18:27:26 +0000 2020,Fri Jan 01 16:24:24 +0000 2010,india,Tweet is personal opinion and Retweet is not e...,False,499,537,2,4342,None


In [12]:
df_users['date'] = pd.to_datetime(df_users['date'])

In [13]:
df_users['twitter_join_date'] = pd.to_datetime(df_users['twitter_join_date'])
df_users.groupby(df_users['twitter_join_date'].dt.year)['twitter_join_date'].count()

twitter_join_date
2006       19
2007     2012
2008     2785
2009    19945
2010    15987
2011    14673
2012    15015
2013    12344
2014    11767
2015    10409
2016    12793
2017    14007
2018    15580
2019    24308
2020    21050
Name: twitter_join_date, dtype: int64

In [14]:
min_date = df_users['date'].min()
max_date = df_users['date'].max()
print(min_date)
print(max_date)

2013-06-03 15:32:48+00:00
2020-04-25 14:48:38+00:00


In [15]:
dates_2020 = df_users.groupby(pd.Grouper(key='date', freq='D'))['date'].count().loc['2020-01-01':'2020-04-25']
dates_2020

date
2020-01-01 00:00:00+00:00         0
2020-01-02 00:00:00+00:00         0
2020-01-03 00:00:00+00:00         0
2020-01-04 00:00:00+00:00         0
2020-01-05 00:00:00+00:00         0
2020-01-06 00:00:00+00:00         0
2020-01-07 00:00:00+00:00         0
2020-01-08 00:00:00+00:00         0
2020-01-09 00:00:00+00:00         0
2020-01-10 00:00:00+00:00         0
2020-01-11 00:00:00+00:00         0
2020-01-12 00:00:00+00:00         0
2020-01-13 00:00:00+00:00         0
2020-01-14 00:00:00+00:00         1
2020-01-15 00:00:00+00:00         0
2020-01-16 00:00:00+00:00         0
2020-01-17 00:00:00+00:00         0
2020-01-18 00:00:00+00:00         0
2020-01-19 00:00:00+00:00         0
2020-01-20 00:00:00+00:00         0
2020-01-21 00:00:00+00:00         0
2020-01-22 00:00:00+00:00         0
2020-01-23 00:00:00+00:00         0
2020-01-24 00:00:00+00:00         0
2020-01-25 00:00:00+00:00         1
2020-01-26 00:00:00+00:00         2
2020-01-27 00:00:00+00:00         0
2020-01-28 00:00:00+00:

In [ ]:
df_users.to_csv('data/users.csv', index=False)

Based on the distribution of the rows in the date column, we can create partitions on the data based on its year

Two indexes were recommended to optimize searching for users by their name, number of followers, and verification status. 
A single column index is used for the "name" column, since it is the primary search criterion. A compound index on the "followers_count" and "verified" columns is created to sort and filter records based on the priority of number of followers and verification status. The "followers_count" is specified first in the index definition with a descending order, since sorting in descending order allows for efficient filtering based on the maximum number of followers. These indexing strategies will help speed up query times while minimizing the impact on write performance.

In [ ]:
# Create an index on the "name" column
cur.execute("CREATE INDEX name_idx_p ON twitter_users_partitioned (name);")
conn.commit()

# Create a compound index on the "followers_count" and "verified" columns
cur.execute("CREATE INDEX followers_verified_idx_p ON twitter_users_partitioned (followers_count DESC, verified);")
conn.commit()

# Close the database connection
#conn.close()

In [ ]:
df_users.head()

In [ ]:
# try:
#     for index, row in users_df.iterrows():
#         cur.execute("""
#             INSERT INTO twitter_users (user_id, name, screen_name, date, twitter_join_date, location, description, verified, followers_count, friends_count, listed_count, favourites_count, preferred_language)
#             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#             ON CONFLICT (user_id) DO UPDATE
#             SET name = excluded.name,
#                 screen_name = excluded.screen_name,
#                 twitter_join_date = excluded.twitter_join_date,
#                 location = excluded.location,
#                 description = excluded.description,
#                 verified = excluded.verified,
#                 followers_count = excluded.followers_count,
#                 friends_count = excluded.friends_count,
#                 listed_count = excluded.listed_count,
#                 favourites_count = excluded.favourites_count,
#                 preferred_language = excluded.preferred_language;
#         """, (row['user_id'], row['name'], row['screen_name'], row['date'], row['twitter_join_date'], row['location'], row['description'], row['verified'], row['followers_count'], row['friends_count'], row['listed_count'], row['favourites_count'], row['preferred_language']))

# except Exception as e:
#     conn.rollback()
#     cur.close()
#     conn.close()
#     print(e)


In [ ]:
pd.read_sql_query('''SELECT EXTRACT(YEAR FROM twitter_join_date) AS year, count(*)
	FROM public.twitter_users group by EXTRACT(YEAR FROM twitter_join_date)
	order by EXTRACT(YEAR FROM twitter_join_date);''', con = conn)